In [1]:
!pip install -q -U kaggle_environments

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.2 MB/s eta 0:00:00


In [2]:
from kaggle_environments import make, evaluate
import random
import pandas as pd

In [11]:
# 1. Бот, который всегда выбирает "Камень"
def rock_bot(observation, configuration):
    """
    Бот, который всегда выбирает "Камень".
    Это простая и предсказуемая стратегия. Противники легко адаптируются к этому боту, но он может
    быть успешен против ботов, которые случайно или часто выбирают "Ножницы".
    """
    return 0  # 0 соответствует "Камень"

In [12]:
# 2. Бот, который всегда выбирает "Бумагу"
def paper_bot(observation, configuration):
    """
    Бот, который всегда выбирает "Бумагу".
    Подобно rock_bot, этот бот придерживается одной стратегии и делает его предсказуемым.
    Он будет успешен против ботов, которые часто выбирают "Камень".
    """
    return 1  # 1 соответствует "Бумага"


In [13]:
# 3. Бот, который всегда выбирает "Ножницы"
def scissors_bot(observation, configuration):
    """
    Бот, который всегда выбирает "Ножницы".
    Третий бот в серии "простых стратегий". Он эффективен против ботов, которые часто выбирают "Бумагу".
    """
    return 2  # 2 соответствует "Ножницы"

In [14]:
# 4. Бот, который выбирает случайное действие
def random_bot(observation, configuration):
    """
    Бот, который выбирает действие случайным образом.
    Случайный выбор делает бота непредсказуемым, но также снижает вероятность выигрыша против ботов с адаптивными стратегиями.
    """
    return random.choice([0, 1, 2])

In [15]:
# 5. Бот, который циклично выбирает "Камень", "Бумагу", "Ножницы"
def cycle_bot(observation, configuration):
    """
    Бот, который циклично переключается между "Камнем", "Бумaгой" и "Ножницами".
    Эта стратегия создает предсказуемую, но не полностью фиксированную последовательность,
    что может работать против более статичных ботов.
    """
    if observation.step == 0:
        cycle_bot.last_move = 0  # Начинаем с "Камня" на первом ходу
    cycle_bot.last_move = (cycle_bot.last_move + 1) % 3  # Переход на следующий ход в цикле
    return cycle_bot.last_move

In [16]:
# 6. Бот, который копирует последний ход противника
def copy_bot(observation, configuration):
    """
    Бот, который повторяет последний ход противника.
    Данная стратегия эффективна против предсказуемых ботов, но слаба против адаптивных стратегий,
    так как копирование ходов не дает преимущества.
    """
    if observation.step == 0:
        return 0  # Начинаем с "Камня" на первом ходу
    return observation.lastOpponentAction


In [17]:
# 7. Бот, который делает контр-ход против последнего хода противника
def counter_bot(observation, configuration):
    """
    Бот, который выбирает ход, чтобы победить последний ход противника.
    Контр-стратегия, которая работает против ботов с повторяющимися действиями, предсказывая и побеждая их последний ход.
    """
    if observation.step == 0:
        return 0  # Начинаем с "Камня" на первом ходу
    return (observation.lastOpponentAction + 1) % 3  # Контр-ход: следующий ход, который побеждает последний ход противника

In [18]:
# 8. Бот, который анализирует частоту ходов противника и выбирает ход, чтобы победить наиболее часто используемый
def frequency_bot(observation, configuration):
    """
    Бот, который анализирует частоту ходов противника и выбирает ход, чтобы победить наиболее часто используемый.
    Хорош против предсказуемых ботов, так как адаптируется к их частотам.
    """
    if observation.step == 0:
        frequency_bot.history = [0, 0, 0]  # Инициализация счётчиков частоты ходов противника
    else:
        frequency_bot.history[observation.lastOpponentAction] += 1  # Увеличиваем счётчик для последнего хода противника

    # Определяем наиболее частый ход противника и выбираем контр-ход
    most_common = frequency_bot.history.index(max(frequency_bot.history))
    return (most_common + 1) % 3  # Ход, который побеждает наиболее частый ход

In [19]:
# 9. Бот, который повторяет свой предыдущий ход с вероятностью 70%
def probabilistic_repeat_bot(observation, configuration):
    """
    Бот, который с большей вероятностью повторяет свой предыдущий ход.
    Вероятность повторения предыдущего хода делает стратегию менее предсказуемой, но все равно склонной к цикличности.
    """
    if observation.step == 0:
        probabilistic_repeat_bot.last_move = random.choice([0, 1, 2])  # Случайный первый ход
    else:
        if random.random() < 0.7:
            return probabilistic_repeat_bot.last_move  # Повторяем прошлый ход с вероятностью 70%
        probabilistic_repeat_bot.last_move = random.choice([0, 1, 2])  # Иначе выбираем случайный ход
    return probabilistic_repeat_bot.last_move

In [20]:
# 10. Бот, который играет зеркально с задержкой на один ход
def delayed_copy_bot(observation, configuration):
    """
    Бот, который копирует ход противника с задержкой на один ход.
    Может сбить с толку некоторых противников, так как имитирует их прошлое действие с небольшой задержкой.
    """
    if observation.step < 2:
        return random.choice([0, 1, 2])  # Случайный выбор на первых двух ходах
    return observation['action'][-2]  # Копируем ход, который противник сделал два хода назад

In [21]:
# 11. Бот, который переключается на ход, чтобы победить свой предыдущий ход
def self_counter_bot(observation, configuration):
    """
    Бот, который переключается на ход, чтобы победить свой предыдущий ход.
    Интересная стратегия, которая стремится контрить саму себя, создавая некую цикличность, но непредсказуемую для противников.
    """
    if observation.step == 0:
        return 0  # Начинаем с "Камня" на первом ходу
    return (observation['action'][-1] + 1) % 3  # Контр-ход против предыдущего собственного хода

In [22]:
# 12. Бот, который случайным образом выбирает между агрессивным и пассивным стилем игры
def adaptive_random_bot(observation, configuration):
    """
    Бот, который случайным образом выбирает между агрессивным и пассивным стилем игры.
    Агрессивный стиль предполагает выбор между "Камнем" и "Ножницами", а пассивный стиль — только "Бумагу".
    """
    if random.random() < 0.5:
        return random.choice([0, 2])  # Агрессивный стиль: Камень или Ножницы
    return 1  # Пассивный стиль: Бумага

In [4]:
# --- Настройка окружения и запуск турнира с evaluate ---

# Определяем список ботов для турнира.

bots = [
    rock_bot,                 # Бот, который всегда выбирает "Камень"
    paper_bot,                # Бот, который всегда выбирает "Бумагу"
    scissors_bot,             # Бот, который всегда выбирает "Ножницы"
    random_bot,               # Бот, который выбирает ход случайным образом
    cycle_bot,                # Бот, который циклично переключается между "Камнем", "Бумaгой" и "Ножницами"
    copy_bot,                 # Бот, который копирует последний ход противника
    counter_bot,              # Бот, который выбирает контр-ход против последнего хода противника
    frequency_bot,            # Бот, который анализирует частоту ходов противника и выбирает контр-ход против самого частого
    probabilistic_repeat_bot, # Бот, который с высокой вероятностью повторяет свой предыдущий ход
    delayed_copy_bot,         # Бот, который копирует ход противника с задержкой на один ход
    self_counter_bot,         # Бот, который выбирает ход, чтобы победить свой предыдущий ход
    adaptive_random_bot       # Бот, который случайным образом переключается между агрессивным и пассивным стилем игры
]

# Список с названиями ботов.
# Этот список используется для удобства отображения результатов, чтобы по названию было понятно, какая стратегия использовалась.
# Порядок имен в `bot_names` соответствует порядку функций в `bots`, поэтому `bot_names[i]` является названием для `bots[i]`.
# Это позволяет нам легко связывать функцию каждого бота с его названием при отображении результатов.

bot_names = [
    "rock_bot",               # Название для бота, который всегда выбирает "Камень"
    "paper_bot",              # Название для бота, который всегда выбирает "Бумагу"
    "scissors_bot",           # Название для бота, который всегда выбирает "Ножницы"
    "random_bot",             # Название для бота, который выбирает ход случайным образом
    "cycle_bot",              # Название для бота, который циклично переключается между "Камнем", "Бумaгой" и "Ножницами"
    "copy_bot",               # Название для бота, который копирует последний ход противника
    "counter_bot",            # Название для бота, который выбирает контр-ход против последнего хода противника
    "frequency_bot",          # Название для бота, который анализирует частоту ходов противника и выбирает контр-ход против самого частого
    "probabilistic_repeat_bot", # Название для бота, который с высокой вероятностью повторяет свой предыдущий ход
    "delayed_copy_bot",       # Название для бота, который копирует ход противника с задержкой на один ход
    "self_counter_bot",       # Название для бота, который выбирает ход, чтобы победить свой предыдущий ход
    "adaptive_random_bot"     # Название для бота, который случайным образом переключается между агрессивным и пассивным стилем игры
]


In [ ]:
# Количество игр в каждом матче
num_games_per_match = 100  # Каждый матч между двумя ботами будет состоять из 100 игр.
# Количество матчей между каждой парой ботов
num_matches_per_pair = 10  # Каждая пара ботов сыграет 10 матчей, чтобы увеличить статистическую значимость результата.

# Инициализируем пустой список для записи результатов каждого матча
results = []

# Двойной цикл для запуска матчей между каждой уникальной парой ботов
for i in range(len(bots)):
    for j in range(i + 1, len(bots)):
        # Запуск 10 матчей между ботами i и j, каждый матч состоит из 100 игр
        scores = evaluate(
            "rps",  # Окружение "Камень-Ножницы-Бумага"
            [bots[i], bots[j]],  # Пара ботов для соревнования
            configuration={"episodeSteps": num_games_per_match},  # Задаем количество игр в каждом матче
            num_episodes=num_matches_per_pair  # Задаем количество матчей
        )

        # Суммируем результаты для bot_1 (первый бот в паре) по всем матчам
        # Проверяем, чтобы результаты не были None
        total_bot_1_score = sum(score[0] for score in scores if score[0] is not None)

        # Суммируем результаты для bot_2 (второй бот в паре) по всем матчам
        total_bot_2_score = sum(score[1] for score in scores if score[1] is not None)

        # Записываем результаты текущей пары ботов в виде словаря
        # Каждый словарь содержит имена ботов, общий счет каждого бота и параметры матча
        results.append({
            "bot_1": bot_names[i],  # Имя первого бота в паре
            "bot_2": bot_names[j],  # Имя второго бота в паре
            "bot_1_total_score": total_bot_1_score,  # Итоговый счет первого бота за все матчи
            "bot_2_total_score": total_bot_2_score,  # Итоговый счет второго бота за все матчи
            "num_matches": num_matches_per_pair,     # Количество матчей
            "games_per_match": num_games_per_match   # Количество игр в каждом матче
        })

# Преобразование списка результатов в DataFrame для удобного отображения и анализа
results_df = pd.DataFrame(results)

In [8]:
# Вывод таблицы результатов
results_df

,bot_1,bot_2,bot_1_total_score,bot_2_total_score,num_matches,games_per_match
0,rock_bot,paper_bot,-990.0,990.0,10,100
1,rock_bot,scissors_bot,990.0,-990.0,10,100
2,rock_bot,random_bot,0.0,0.0,10,100
3,rock_bot,cycle_bot,0.0,0.0,10,100
4,rock_bot,copy_bot,0.0,0.0,10,100
...,...,...,...,...,...,...
61,probabilistic_repeat_bot,self_counter_bot,10.0,0.0,10,100
62,probabilistic_repeat_bot,adaptive_random_bot,0.0,0.0,10,100
63,delayed_copy_bot,self_counter_bot,10.0,0.0,10,100
64,delayed_copy_bot,adaptive_random_bot,0.0,10.0,10,100


In [9]:
# Подсчет очков для каждого бота
# Создаем пустой словарь total_scores, чтобы хранить общий счет каждого бота по итогам турнира.
total_scores = {}

# Проходим по каждой строке таблицы results_df, где хранится результат каждого матча между парами ботов.
for index, row in results_df.iterrows():
    # Извлекаем имена ботов и их итоговые очки за матч
    bot_1 = row['bot_1']  # Имя первого бота в текущей паре
    bot_2 = row['bot_2']  # Имя второго бота в текущей паре
    bot_1_score = row['bot_1_total_score']  # Итоговый счет первого бота за данный матч
    bot_2_score = row['bot_2_total_score']  # Итоговый счет второго бота за данный матч

    # Обновляем общий счет для bot_1
    # Если бот отсутствует в словаре total_scores, добавляем его с начальным значением 0
    if bot_1 not in total_scores:
        total_scores[bot_1] = 0
    # Добавляем очки за текущий матч к общему счету бота
    total_scores[bot_1] += bot_1_score

    # Обновляем общий счет для bot_2 (аналогично bot_1)
    if bot_2 not in total_scores:
        total_scores[bot_2] = 0
    total_scores[bot_2] += bot_2_score

# Преобразуем total_scores в DataFrame для удобного просмотра
# total_scores хранит общие очки для каждого бота в виде словаря: ключ — имя бота, значение — общий счет
final_scores_df = pd.DataFrame(list(total_scores.items()), columns=['Bot', 'Total_Score'])

# Сортируем ботов по их общему счету в порядке убывания, чтобы лучшие боты оказались в начале таблицы
final_scores_df = final_scores_df.sort_values(by='Total_Score', ascending=False).reset_index(drop=True)

In [10]:
# Вывод таблицы итоговых результатов
final_scores_df

,Bot,Total_Score
0,counter_bot,4603.0
1,frequency_bot,2621.0
2,cycle_bot,1340.0
3,random_bot,20.0
4,delayed_copy_bot,10.0
5,self_counter_bot,0.0
6,adaptive_random_bot,-186.0
7,probabilistic_repeat_bot,-650.0
8,copy_bot,-1670.0
9,scissors_bot,-1783.0


Эта таблица показывает **итоговые очки для каждого бота** по результатам всех матчей в турнире.

1. **Ранжирование ботов**: Боты отсортированы по их общему счету (`Total_Score`). Чем выше счет, тем лучше бот выступил в турнире.
   
2. **Лидеры турнира**:
   - **counter_bot** — набрал наибольшее количество очков (4603), что говорит о его высокой эффективности в противостоянии с другими ботами. Этот бот выбирает ход, который побеждает последний ход противника, что помогает ему адаптироваться к стратегиям, повторяющимся или легко предсказуемым.
   - **frequency_bot** (2621) — второй по результатам. Он анализирует частоту ходов противника и выбирает ход, чтобы победить наиболее часто используемый ход. Такая стратегия эффективна против ботов, которые склонны к предсказуемым действиям.
   - **cycle_bot** (1340) — третий по результатам. Циклический бот переключается между "Камень", "Бумага", "Ножницы" по очереди, что делает его немного менее предсказуемым, чем боты, выбирающие одно и то же действие постоянно.

3. **Аутсайдеры**:
   - **rock_bot**, **paper_bot**, и **scissors_bot** — боты с отрицательными итоговыми очками. Эти боты всегда выбирают одно и то же действие, что делает их предсказуемыми и легкими для победы. Они занимают последние позиции, так как большинство других ботов могут легко адаптироваться к их стратегии.
   - **copy_bot** и **probabilistic_repeat_bot** также находятся в нижней части списка. Их стратегии оказались менее успешными против более адаптивных и гибких ботов, которые могут предсказать их поведение.

## Заключение:

1. **Адаптивные стратегии более успешны**: Боты, которые могут адаптироваться к действиям противника, такие как `counter_bot` и `frequency_bot`, показали себя лучше всего. Эти боты анализируют поведение противника и используют полученную информацию для выбора оптимального хода.
   
2. **Статичные стратегии проигрывают**: Боты, которые выбирают один и тот же ход каждый раз (`rock_bot`, `paper_bot`, `scissors_bot`), показали худшие результаты. Это подчеркивает важность адаптации в игре «Камень-Ножницы-Бумага».

3. **Случайные стратегии показывают средние результаты**: `random_bot` и другие случайные стратегии показали результаты, близкие к нулю, что указывает на то, что случайный выбор не является оптимальной стратегией, но также не приводит к полному провалу.